# Chapter 13: Practical Cryptography Applications

In this interactive chapter, we'll implement and explore various cryptographic algorithms using Python. You can run these examples directly in your browser!

## Setup

First, let's import the libraries we'll need:

In [ ]:
# Import necessary libraries
import hashlib
import secrets
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.backends import default_backend
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
import base64

print("Libraries imported successfully!")

## 1. Classical Ciphers

### Caesar Cipher Implementation

In [ ]:
def caesar_encrypt(plaintext, shift):
    """Encrypt plaintext using Caesar cipher with given shift."""
    ciphertext = ""
    for char in plaintext.upper():
        if char.isalpha():
            # Shift character and wrap around alphabet
            shifted = chr(((ord(char) - ord('A') + shift) % 26) + ord('A'))
            ciphertext += shifted
        else:
            ciphertext += char
    return ciphertext

def caesar_decrypt(ciphertext, shift):
    """Decrypt ciphertext using Caesar cipher with given shift."""
    return caesar_encrypt(ciphertext, -shift)

# Example usage
plaintext = "HELLO WORLD"
shift = 3
ciphertext = caesar_encrypt(plaintext, shift)
decrypted = caesar_decrypt(ciphertext, shift)

print(f"Plaintext:  {plaintext}")
print(f"Ciphertext: {ciphertext}")
print(f"Decrypted:  {decrypted}")

```{admonition} Try It Yourself!
:class: tip
Modify the code above to:
1. Try different shift values
2. Encrypt your own message
3. Implement a brute-force attack to break the cipher
```

### Vigenère Cipher

In [ ]:
def vigenere_encrypt(plaintext, key):
    """Encrypt plaintext using Vigenère cipher."""
    ciphertext = ""
    key = key.upper()
    key_index = 0
    
    for char in plaintext.upper():
        if char.isalpha():
            # Get shift from key
            shift = ord(key[key_index % len(key)]) - ord('A')
            # Encrypt character
            encrypted = chr(((ord(char) - ord('A') + shift) % 26) + ord('A'))
            ciphertext += encrypted
            key_index += 1
        else:
            ciphertext += char
    return ciphertext

def vigenere_decrypt(ciphertext, key):
    """Decrypt ciphertext using Vigenère cipher."""
    plaintext = ""
    key = key.upper()
    key_index = 0
    
    for char in ciphertext.upper():
        if char.isalpha():
            # Get shift from key
            shift = ord(key[key_index % len(key)]) - ord('A')
            # Decrypt character
            decrypted = chr(((ord(char) - ord('A') - shift) % 26) + ord('A'))
            plaintext += decrypted
            key_index += 1
        else:
            plaintext += char
    return plaintext

# Example
plaintext = "CRYPTOGRAPHY IS FUN"
key = "SECRETKEY"
ciphertext = vigenere_encrypt(plaintext, key)
decrypted = vigenere_decrypt(ciphertext, key)

print(f"Plaintext:  {plaintext}")
print(f"Key:        {key}")
print(f"Ciphertext: {ciphertext}")
print(f"Decrypted:  {decrypted}")

## 2. Hash Functions

### SHA-256 Example

In [ ]:
def compute_hash(message):
    """Compute SHA-256 hash of a message."""
    # Convert message to bytes
    message_bytes = message.encode('utf-8')
    
    # Create hash object
    hash_object = hashlib.sha256(message_bytes)
    
    # Get hexadecimal representation
    hex_digest = hash_object.hexdigest()
    
    return hex_digest

# Example
message1 = "Hello, World!"
message2 = "Hello, World?"  # Slightly different

hash1 = compute_hash(message1)
hash2 = compute_hash(message2)

print(f"Message 1: {message1}")
print(f"Hash 1:    {hash1}\n")

print(f"Message 2: {message2}")
print(f"Hash 2:    {hash2}\n")

print(f"Hashes are different: {hash1 != hash2}")

```{note}
Notice how a tiny change in the input (! vs ?) produces a completely different hash. This is the **avalanche effect**.
```

## 3. AES Encryption

### AES in CBC Mode

In [ ]:
def aes_cbc_encrypt(plaintext, key):
    """Encrypt plaintext using AES in CBC mode."""
    # Generate random IV
    iv = get_random_bytes(16)
    
    # Create cipher object
    cipher = AES.new(key, AES.MODE_CBC, iv)
    
    # Pad plaintext to multiple of 16 bytes
    padded_plaintext = pad(plaintext.encode('utf-8'), AES.block_size)
    
    # Encrypt
    ciphertext = cipher.encrypt(padded_plaintext)
    
    # Return IV + ciphertext (both needed for decryption)
    return iv + ciphertext

def aes_cbc_decrypt(ciphertext_with_iv, key):
    """Decrypt ciphertext using AES in CBC mode."""
    # Extract IV and ciphertext
    iv = ciphertext_with_iv[:16]
    ciphertext = ciphertext_with_iv[16:]
    
    # Create cipher object
    cipher = AES.new(key, AES.MODE_CBC, iv)
    
    # Decrypt and unpad
    padded_plaintext = cipher.decrypt(ciphertext)
    plaintext = unpad(padded_plaintext, AES.block_size)
    
    return plaintext.decode('utf-8')

# Example
# Generate a random 256-bit (32-byte) key
key = get_random_bytes(32)

plaintext = "This is a secret message that needs to be encrypted!"
print(f"Original plaintext: {plaintext}")

# Encrypt
encrypted = aes_cbc_encrypt(plaintext, key)
print(f"\nEncrypted (base64): {base64.b64encode(encrypted).decode('utf-8')}")

# Decrypt
decrypted = aes_cbc_decrypt(encrypted, key)
print(f"\nDecrypted plaintext: {decrypted}")

print(f"\nDecryption successful: {plaintext == decrypted}")

## 4. Password Hashing with PBKDF2

In [ ]:
def hash_password(password, salt=None, iterations=100000):
    """Hash a password using PBKDF2."""
    if salt is None:
        salt = secrets.token_bytes(16)
    
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=iterations,
        backend=default_backend()
    )
    
    key = kdf.derive(password.encode('utf-8'))
    
    # Return salt + hash for storage
    return salt + key

def verify_password(password, stored_hash):
    """Verify a password against a stored hash."""
    # Extract salt from stored hash
    salt = stored_hash[:16]
    
    # Hash the provided password with the same salt
    new_hash = hash_password(password, salt)
    
    # Compare hashes
    return new_hash == stored_hash

# Example
password = "MySecurePassword123!"
print(f"Password: {password}")

# Hash the password
stored_hash = hash_password(password)
print(f"\nStored hash (base64): {base64.b64encode(stored_hash).decode('utf-8')}")

# Verify correct password
is_correct = verify_password(password, stored_hash)
print(f"\nPassword verification (correct): {is_correct}")

# Verify incorrect password
is_incorrect = verify_password("WrongPassword", stored_hash)
print(f"Password verification (incorrect): {is_incorrect}")

```{important}
**Why PBKDF2?**
- Adds computational cost (iterations)
- Uses salt to prevent rainbow table attacks
- Industry standard for password storage
```

## 5. Message Authentication Code (HMAC)

In [ ]:
def compute_hmac(message, key):
    """Compute HMAC-SHA256 of a message."""
    h = hmac.HMAC(key, hashes.SHA256(), backend=default_backend())
    h.update(message.encode('utf-8'))
    return h.finalize()

def verify_hmac(message, key, tag):
    """Verify HMAC-SHA256 tag."""
    h = hmac.HMAC(key, hashes.SHA256(), backend=default_backend())
    h.update(message.encode('utf-8'))
    try:
        h.verify(tag)
        return True
    except:
        return False

# Example
key = secrets.token_bytes(32)
message = "Authenticate this message"

# Compute HMAC
tag = compute_hmac(message, key)
print(f"Message: {message}")
print(f"HMAC tag (base64): {base64.b64encode(tag).decode('utf-8')}")

# Verify with correct message
is_valid = verify_hmac(message, key, tag)
print(f"\nVerification (correct message): {is_valid}")

# Verify with tampered message
tampered_message = "Authenticate this MESSAGE"  # Changed to uppercase
is_invalid = verify_hmac(tampered_message, key, tag)
print(f"Verification (tampered message): {is_invalid}")

## 6. Secure Random Number Generation

In [ ]:
# Generate cryptographically secure random data

# Random bytes
random_bytes = secrets.token_bytes(32)
print(f"Random bytes (hex): {random_bytes.hex()}")

# Random integer
random_int = secrets.randbelow(1000000)
print(f"\nRandom integer (0-999999): {random_int}")

# Random token (URL-safe)
random_token = secrets.token_urlsafe(32)
print(f"\nRandom token (URL-safe): {random_token}")

# Generate a secure password
import string
alphabet = string.ascii_letters + string.digits + string.punctuation
password = ''.join(secrets.choice(alphabet) for i in range(16))
print(f"\nGenerated password: {password}")

```{warning}
**Never use `random` module for cryptography!**

Always use `secrets` module or cryptographic libraries for:
- Keys
- Tokens
- Nonces
- Passwords
- IVs
```

## Summary

In this interactive chapter, you learned:

1. ✅ **Classical Ciphers**: Caesar and Vigenère implementations
2. ✅ **Hash Functions**: SHA-256 and the avalanche effect
3. ✅ **AES Encryption**: Symmetric encryption with CBC mode
4. ✅ **Password Hashing**: PBKDF2 for secure password storage
5. ✅ **Message Authentication**: HMAC for integrity verification
6. ✅ **Secure Random**: Cryptographically secure random generation

## Exercises

```{exercise}
:label: implement-affine
Implement the Affine cipher (encryption and decryption) in Python.
```

```{exercise}
:label: frequency-analysis-code
Write a function to perform frequency analysis on English text to help break substitution ciphers.
```

```{exercise}
:label: aes-gcm
Modify the AES example to use GCM mode instead of CBC mode. Compare the differences.
```

## Further Exploration

Try experimenting with:
- Different key sizes
- Various cipher modes
- Performance comparisons
- Real-world applications

```{admonition} Resources
:class: tip
- [Cryptography Python Library Documentation](https://cryptography.io/)
- [PyCryptodome Documentation](https://pycryptodome.readthedocs.io/)
```